In [2]:
# from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from datetime import *
from sklearn.preprocessing import MinMaxScaler
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.fft as fft
import os
from tqdm import tqdm # 打印进度条
import math
from einops import rearrange, repeat, reduce
from scipy.fftpack import next_fast_len
import matplotlib.pyplot as plt
import functools
from functools import partial, wraps
from sympy import Poly, legendre, Symbol, chebyshevt
from scipy.special import eval_legendre
from scipy import signal
from torch.nn.modules.linear import Linear
from operator import mul
from typing import List
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset
from sklearn.metrics import accuracy_score
import onnx
import onnxruntime as ort
import pandas as pd
import joblib
# 两种绘图接口
import plotly_express as px
import plotly.graph_objects as go
from sklearn.metrics import precision_recall_fscore_support

import warnings

warnings.filterwarnings("ignore")

In [115]:
data_path = "../data/UWaveGestureLibraryAll_TRAIN.tsv"
data = pd.read_csv(data_path, sep='\t', header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,936,937,938,939,940,941,942,943,944,945
0,6,-0.312502,-0.312502,-0.312502,-0.312502,-0.312502,-0.312502,-0.312502,-0.312502,-0.312502,...,0.512194,0.503002,0.492524,0.482045,0.463662,0.445278,0.426895,0.408511,0.390129,0.371746
1,5,1.629721,1.629721,1.629721,1.629721,1.629721,1.629721,1.629721,1.629721,1.629721,...,-0.424252,-0.424252,-0.424252,-0.424414,-0.426015,-0.437960,-0.449905,-0.461849,-0.473794,-0.485739
2,5,0.665410,0.665410,0.665410,0.665410,0.665410,0.665410,0.665410,0.665410,0.665410,...,-0.856895,-0.856895,-0.856895,-0.856895,-0.856895,-0.856895,-0.856895,-0.856895,-0.856895,-0.856895
3,3,0.164887,0.164887,0.164887,0.164887,0.164887,0.164887,0.164887,0.164887,0.164887,...,-0.194748,-0.131015,-0.286569,0.051284,0.389136,0.149143,0.245672,0.437129,0.639840,0.842551
4,4,1.272493,1.272493,1.272493,1.272493,1.272493,1.272493,1.272493,1.272493,1.272493,...,1.852405,1.819330,1.741195,1.624052,1.506909,1.439170,1.500978,1.618121,1.735263,1.852405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,3,0.126056,0.126056,0.126056,0.126056,0.126056,0.126056,0.126056,0.126056,0.126056,...,-0.202822,-0.128241,-0.321706,-0.044231,0.142222,0.328674,0.110026,0.147317,0.184607,0.221897
892,1,-1.232312,-1.232312,-1.232312,-1.232312,-1.232312,-1.232312,-1.232312,-1.232312,-1.232312,...,0.221684,0.233878,0.246070,0.258264,0.270456,0.282650,0.294843,0.307036,0.319229,0.331423
893,2,0.285103,0.285103,0.285103,0.285103,0.285103,0.285103,0.285103,0.285103,0.285103,...,-0.219075,-0.213164,-0.187128,0.176418,0.138347,0.198371,0.446330,0.161218,0.225716,0.270101
894,7,1.251416,1.251416,1.251416,1.251416,1.251416,1.251416,1.251416,1.251416,1.251416,...,-0.155403,-0.158187,-0.185514,-0.291492,-0.401972,-0.499320,-0.553890,-0.544005,-0.500638,-0.427876


In [116]:
data1 = data
data2 = data

In [128]:
load_all([data1], ['feature1'])

(                                              feature1
 0    [-0.31250227, -0.31250227, -0.31250227, -0.312...
 1    [1.6297212, 1.6297212, 1.6297212, 1.6297212, 1...
 2    [0.66540969, 0.66540969, 0.66540969, 0.6654096...
 3    [0.16488744, 0.16488744, 0.16488744, 0.1648874...
 4    [1.2724935, 1.2724935, 1.2724935, 1.2724935, 1...
 ..                                                 ...
 891  [0.12605618, 0.12605618, 0.12605618, 0.1260561...
 892  [-1.2323122, -1.2323122, -1.2323122, -1.232312...
 893  [0.28510341, 0.28510341, 0.28510341, 0.2851034...
 894  [1.2514163, 1.2514163, 1.2514163, 1.2514163, 1...
 895  [-0.16639664, -0.16639664, -0.16639664, -0.166...
 
 [896 rows x 1 columns],
 array([6, 5, 5, 3, 4, 8, 7, 4, 4, 6, 1, 7, 3, 5, 2, 6, 1, 6, 1, 2, 8, 6,
        1, 7, 8, 7, 3, 6, 2, 6, 2, 7, 8, 4, 5, 6, 8, 1, 7, 8, 5, 6, 1, 6,
        5, 1, 7, 8, 1, 5, 4, 4, 7, 1, 4, 1, 8, 2, 1, 2, 3, 8, 1, 4, 2, 1,
        1, 7, 5, 5, 2, 5, 7, 2, 1, 7, 1, 6, 5, 3, 7, 2, 1, 6, 2, 7, 1, 1,
     

In [83]:
df1 = data1.iloc[:, 1:]
# row.tolist()将这一行的所有列值提取并转换为一个普通的 Python 列表
df1 = pd.DataFrame(df1.apply(lambda row: row.tolist(), axis=1),columns=['feature1'])

In [84]:
df2 = data2.iloc[:, 1:]
# row.tolist()将这一行的所有列值提取并转换为一个普通的 Python 列表
df2 = pd.DataFrame(df2.apply(lambda row: row.tolist(), axis=1),columns=['feature2'])

,feature2
0,"[-0.31250227, -0.31250227, -0.31250227, -0.312..."
1,"[1.6297212, 1.6297212, 1.6297212, 1.6297212, 1..."
2,"[0.66540969, 0.66540969, 0.66540969, 0.6654096..."
3,"[0.16488744, 0.16488744, 0.16488744, 0.1648874..."
4,"[1.2724935, 1.2724935, 1.2724935, 1.2724935, 1..."
...,...
891,"[0.12605618, 0.12605618, 0.12605618, 0.1260561..."
892,"[-1.2323122, -1.2323122, -1.2323122, -1.232312..."
893,"[0.28510341, 0.28510341, 0.28510341, 0.2851034..."
894,"[1.2514163, 1.2514163, 1.2514163, 1.2514163, 1..."


In [87]:
df = pd.concat([df1, df2], axis=1)
df

In [88]:
labels = data.iloc[:, 0].values
labels

array([6, 5, 5, 3, 4, 8, 7, 4, 4, 6, 1, 7, 3, 5, 2, 6, 1, 6, 1, 2, 8, 6,
       1, 7, 8, 7, 3, 6, 2, 6, 2, 7, 8, 4, 5, 6, 8, 1, 7, 8, 5, 6, 1, 6,
       5, 1, 7, 8, 1, 5, 4, 4, 7, 1, 4, 1, 8, 2, 1, 2, 3, 8, 1, 4, 2, 1,
       1, 7, 5, 5, 2, 5, 7, 2, 1, 7, 1, 6, 5, 3, 7, 2, 1, 6, 2, 7, 1, 1,
       5, 6, 4, 8, 6, 3, 4, 5, 3, 8, 3, 5, 2, 8, 8, 8, 8, 7, 5, 2, 2, 5,
       2, 3, 4, 4, 7, 7, 1, 4, 5, 4, 1, 6, 5, 5, 4, 6, 4, 6, 2, 3, 6, 6,
       3, 3, 5, 5, 3, 6, 5, 8, 4, 6, 1, 8, 5, 8, 2, 5, 5, 8, 2, 7, 1, 2,
       4, 2, 3, 8, 7, 7, 6, 4, 3, 1, 3, 4, 3, 8, 6, 2, 7, 3, 5, 2, 2, 2,
       4, 5, 7, 6, 1, 1, 8, 6, 6, 1, 6, 1, 3, 2, 7, 5, 2, 5, 5, 8, 7, 4,
       7, 5, 8, 4, 2, 4, 1, 1, 6, 4, 5, 3, 2, 8, 4, 6, 3, 4, 5, 1, 8, 6,
       2, 4, 6, 1, 8, 8, 7, 4, 4, 2, 2, 1, 4, 8, 5, 6, 1, 5, 3, 3, 4, 3,
       6, 3, 6, 1, 1, 3, 7, 2, 5, 2, 3, 6, 8, 2, 4, 8, 6, 7, 4, 1, 6, 3,
       7, 6, 5, 4, 1, 7, 7, 1, 4, 7, 8, 5, 3, 8, 5, 7, 7, 7, 5, 5, 4, 1,
       8, 7, 3, 3, 4, 7, 5, 6, 1, 6, 4, 8, 7, 7, 6,

In [129]:
class TensorDataset(Dataset):
    def __init__(self, data, columns):
        df, labels = self.load_all(data, columns)
        self.all_df, self.labels_df = self.load_single(df, labels)
        self.all_IDs = self.all_df.index.unique()  # all sample IDs (integer indices 0 ... num_samples-1)

        # use all features
        self.feature_names = self.all_df.columns
        self.feature_df = self.all_df

        # pre_process
#         normalizer = Normalizer()
#         self.feature_df = normalizer.normalize(self.feature_df)
#         print(len(self.all_IDs))
    # 加载多个数据集
    def load_all(self, data: List[pd.DataFrame], columns: List[str]):
        # 特征数据合并
        for i in range(len(data)):
            if i == 0:
                df = data[i].iloc[:, 1:]
                # row.tolist()将这一行的所有列值提取并转换为一个普通的 Python 列表
                df = pd.DataFrame(df.apply(lambda row: row.tolist(), axis=1),
                                  columns=[columns[i]])
            else:
                df2 = data[i].iloc[:, 1:]
                df2 = pd.DataFrame(df2.apply(lambda row: row.tolist(), axis=1),
                                   columns=[columns[i]])
                df = pd.concat([df, df2], axis=1)

        # 标签数据
        labels = data[0].iloc[:, 0].values

        return df, labels

    # 加载单个数据集
    def load_single(self, df, labels):
        labels = pd.Series(labels, dtype="category")
        self.class_names = labels.cat.categories
        labels_df = pd.DataFrame(labels.cat.codes,
                                 dtype=np.int8)  # int8-32 gives an error when using nn.CrossEntropyLoss

        lengths = df.applymap(
            lambda x: len(x)).values  # (num_samples, num_dimensions) array containing the length of each series

        horiz_diffs = np.abs(lengths - np.expand_dims(lengths[:, 0], -1))

        if np.sum(horiz_diffs) > 0:  # if any row (sample) has varying length across dimensions
            df = df.applymap(self.subsample)

        lengths = df.applymap(lambda x: len(x)).values
        vert_diffs = np.abs(lengths - np.expand_dims(lengths[0, :], 0))
        if np.sum(vert_diffs) > 0:  # if any column (dimension) has varying length across samples
            self.max_seq_len = int(np.max(lengths[:, 0]))
        else:
            self.max_seq_len = lengths[0, 0]

        # First create a (seq_len, feat_dim) dataframe for each sample, indexed by a single integer ("ID" of the sample)
        # Then concatenate into a (num_samples * seq_len, feat_dim) dataframe, with multiple rows corresponding to the
        # sample index (i.e. the same scheme as all datasets in this project)

        df = pd.concat((pd.DataFrame({col: df.loc[row, col] for col in df.columns}).reset_index(drop=True).set_index(
            pd.Series(lengths[row, 0] * [row])) for row in range(df.shape[0])), axis=0)

        # Replace NaN values
        grp = df.groupby(by=df.index)
        df = grp.transform(self.interpolate_missing)

        return df, labels_df
    
    # 通过采样保持序列长度一致
    def subsample(self, y, limit=256, factor=2):
        """
        If a given Series is longer than `limit`, returns subsampled sequence by the specified integer factor
        """
        if len(y) > limit:
            return y[::factor].reset_index(drop=True)
        return y
    
    # 线性插值填充缺失值
    def interpolate_missing(self, y):
        """
        Replaces NaN values in pd.Series `y` using linear interpolation
        """
        if y.isna().any():
            y = y.interpolate(method='linear', limit_direction='both')
        return y

#     def instance_norm(self, case):
#         if self.root_path.count('EthanolConcentration') > 0:  # special process for numerical stability
#             mean = case.mean(0, keepdim=True)
#             case = case - mean
#             stdev = torch.sqrt(torch.var(case, dim=1, keepdim=True, unbiased=False) + 1e-5)
#             case /= stdev
#             return case
#         else:
#             return case

    def __getitem__(self, ind):
        batch_x = self.feature_df.loc[self.all_IDs[ind]].values
#         print(batch_x.shape)
        labels = self.labels_df.loc[self.all_IDs[ind]].values
#         if self.flag == "TRAIN" and self.args.augmentation_ratio > 0:
    #         # 数据增强，仅限训练阶段
    #         num_samples = len(self.all_IDs)
    #         num_columns = self.feature_df.shape[1]
    #         seq_len = int(self.feature_df.shape[0] / num_samples)
    #         batch_x = batch_x.reshape((1, seq_len, num_columns))
    #         batch_x, labels, augmentation_tags = run_augmentation_single(batch_x, labels, self.args)

        batch_x = batch_x.reshape((1 * seq_len, num_columns))

        return torch.from_numpy(batch_x), \
               torch.from_numpy(labels)

    def __len__(self):
        return len(self.all_IDs)

In [130]:
data_set = TensorDataset([data1, data2], ['feature1', 'feature2'])

In [131]:
def padding_mask(lengths, max_len=None):
    """
    Used to mask padded positions: creates a (batch_size, max_len) boolean mask from a tensor of sequence lengths,
    where 1 means keep element at this position (time step)
    """
    batch_size = lengths.numel()
    max_len = max_len or lengths.max()  # trick works because of overloading of 'or' operator for non-boolean types
    return (torch.arange(0, max_len, device=lengths.device)
            .type_as(lengths)
            .repeat(batch_size, 1)
            .lt(lengths.unsqueeze(1)))

def collate_fn(data, max_len=None):
    """Build mini-batch tensors from a list of (X, mask) tuples. Mask input. Create
    Args:
        data: len(batch_size) list of tuples (X, y).
            - X: torch tensor of shape (seq_length, feat_dim); variable seq_length.
            - y: torch tensor of shape (num_labels,) : class indices or numerical targets
                (for classification or regression, respectively). num_labels > 1 for multi-task models
        max_len: global fixed sequence length. Used for architectures requiring fixed length input,
            where the batch length cannot vary dynamically. Longer sequences are clipped, shorter are padded with 0s
    Returns:
        X: (batch_size, padded_length, feat_dim) torch tensor of masked features (input)
        targets: (batch_size, padded_length, feat_dim) torch tensor of unmasked features (output)
        target_masks: (batch_size, padded_length, feat_dim) boolean torch tensor
            0 indicates masked values to be predicted, 1 indicates unaffected/"active" feature values
        padding_masks: (batch_size, padded_length) boolean tensor, 1 means keep vector at this position, 0 means padding
    """

    batch_size = len(data)
    features, labels = zip(*data)

    print(features)
    # Stack and pad features and masks (convert 2D to 3D tensors, i.e. add batch dimension)
    lengths = [X.shape[0] for X in features]  # original sequence length for each time series
    if max_len is None:
        max_len = max(lengths)

    X = torch.zeros(batch_size, max_len, features[0].shape[-1])  # (batch_size, padded_length, feat_dim)
    for i in range(batch_size):
        end = min(lengths[i], max_len)
        X[i, :end, :] = features[i][:end, :]

    targets = torch.stack(labels, dim=0)  # (batch_size, num_labels)

#     print(torch.tensor(lengths, dtype=torch.int16).max())
    padding_masks = padding_mask(torch.tensor(lengths, dtype=torch.int16),
                                 max_len=max_len)  # (batch_size, padded_length) boolean tensor, "1" means keep

    return X, targets, padding_masks

In [132]:
data_loader = DataLoader(
            data_set,
            batch_size=16,
            shuffle=True,
            drop_last=False,
            collate_fn=lambda x: collate_fn(x, max_len=data_set.max_seq_len)
        )

In [133]:
for batch_x, batch_y, mask in data_loader:
    print(batch_x.shape)
    print(batch_y.shape)
    print(mask.shape)

NameError: name 'seq_len' is not defined